In [1]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc = MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')
import pandas as pd

# grab the StructureTree instance
structure_tree = mcc.get_structure_tree()

In [2]:
all_edges = pd.read_csv('./whole brain/edge list - projection volume.csv')

In [3]:
# Get list of all nodes involved in feeding from old feeding doc
feeding = pd.read_excel('../../spreadsheets/Experiment Nodes Feeding List.xlsx')
feeding_nodes = feeding.loc[feeding['Associated with feeding?'].str.contains('Y', na=False)][['structure-id', 'structure-abbrev', 'structure-name']]

### Create list of all feeding edges

In [4]:
# Create new dataframe with only feeding edges
feeding_edges = all_edges.loc[(all_edges['origin_id'].isin(feeding_nodes['structure-id'])) | (all_edges['target_id'].isin(feeding_nodes['structure-id']))]
print('There are %d edges involving the feeding circuit.' % feeding_edges.shape[0])
feeding_edges.head()

There are 4773 edges involving the feeding circuit.


,experiment_id,id,origin_abbrev,origin_id,target_abbrev,target_id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,...,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
3,176886238,630101077,BST,351,PT,15,True,1.000000,5740,4620,...,0.117493,0.999045,18104.446380,18121.748533,0.065085,9.631659e+11,5.318124e+07,9.628168e+11,5.313046e+07,0.065147
6,127711803,633842902,CP,672,CEA,536,True,0.141788,6190,5410,...,0.000087,1.000000,17567.098039,17567.098039,0.000006,7.980640e+07,4.542947e+03,7.980640e+07,4.542947e+03,0.000006
10,603890605,641506553,TEa,541,TEa5,289,True,0.123172,8820,2340,...,0.000073,1.000000,17034.916362,17034.916362,0.000003,3.739768e+07,2.195354e+03,3.739768e+07,2.195354e+03,0.000003
13,609157409,641551186,PL,972,PL1,171,True,1.000000,3060,2930,...,0.007497,0.999882,16573.292891,16575.244074,0.001816,2.457081e+10,1.482373e+06,2.456780e+10,1.482198e+06,0.001816
14,176886238,630101459,BST,351,MTN,571,True,1.000000,5850,5070,...,0.159567,0.997939,16362.817350,16396.608046,0.088392,1.183618e+12,7.230566e+07,1.183124e+12,7.215665e+07,0.088574


# Top 50% projection volume analysis

### Select top 50% of edges

In [14]:
# Get top 50% of feeding edges
feeding_edges_50 = feeding_edges.iloc[:round(feeding_edges.shape[0] / 2)]
print('There are %d edges in the top 50%% volumes for feeding' % feeding_edges_50.shape[0])
feeding_edges_50.head()

There are 2386 edges in the top 50% intensities for feeding


,experiment_id,id,origin_abbrev,origin_id,target_abbrev,target_id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,...,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
3,176886238,630101077,BST,351,PT,15,True,1.000000,5740,4620,...,0.117493,0.999045,18104.446380,18121.748533,0.065085,9.631659e+11,5.318124e+07,9.628168e+11,5.313046e+07,0.065147
6,127711803,633842902,CP,672,CEA,536,True,0.141788,6190,5410,...,0.000087,1.000000,17567.098039,17567.098039,0.000006,7.980640e+07,4.542947e+03,7.980640e+07,4.542947e+03,0.000006
10,603890605,641506553,TEa,541,TEa5,289,True,0.123172,8820,2340,...,0.000073,1.000000,17034.916362,17034.916362,0.000003,3.739768e+07,2.195354e+03,3.739768e+07,2.195354e+03,0.000003
13,609157409,641551186,PL,972,PL1,171,True,1.000000,3060,2930,...,0.007497,0.999882,16573.292891,16575.244074,0.001816,2.457081e+10,1.482373e+06,2.456780e+10,1.482198e+06,0.001816
14,176886238,630101459,BST,351,MTN,571,True,1.000000,5850,5070,...,0.159567,0.997939,16362.817350,16396.608046,0.088392,1.183618e+12,7.230566e+07,1.183124e+12,7.215665e+07,0.088574


#### Create 3 groups of feeding network edges
Outgoing edges : Edges which start *in* the network and *leave*

Incoming edges : Edges which start *outside* the network and *enter*

Internal edges : Edges that start *in* the network and *end there*

In [15]:
# Find list of outgoing/incoming/internal edges
outgoing_edges = feeding_edges.loc[~feeding_edges['target_id'].isin(feeding_nodes['structure-id'])]  # The target is not in the feeding network
incoming_edges = feeding_edges.loc[(~feeding_edges['origin_id'].isin(feeding_nodes['structure-id'])) & (feeding_edges['target_id'].isin(feeding_nodes['structure-id']))]  # The origin isn't in the feeding network and the target is
internal_edges = feeding_edges.loc[(feeding_edges['origin_id'].isin(feeding_nodes['structure-id'])) & (feeding_edges['target_id'].isin(feeding_nodes['structure-id']))]  # Both the origin and target are in the feeding network

external_edges = pd.concat([outgoing_edges, incoming_edges])

In [22]:
internal_edges

,experiment_id,id,origin_abbrev,origin_id,target_abbrev,target_id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,...,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
97,175072921,629939668,MEA,403,AAA,23,True,1.000000,5860,6230,...,0.090480,0.999524,15135.384263,15142.590346,0.075367,9.317441e+11,6.155310e+07,9.316298e+11,6.152380e+07,7.540254e-02
134,287769286,632003089,ORBvl,746,MOB,507,True,0.854233,2470,3990,...,0.000760,0.999994,14827.568375,14827.656552,0.000225,2.727137e+09,1.839223e+05,2.727121e+09,1.839212e+05,2.253049e-04
266,180709230,630418124,ORBl,723,AIv,119,True,1.000000,3170,4600,...,0.328827,0.999153,14243.600631,14255.673137,0.262120,3.050814e+12,2.141566e+08,3.050362e+12,2.139753e+08,2.623419e-01
348,263106036,630743599,PL,972,ORBm,731,True,1.000000,3060,3470,...,0.165183,0.999955,13973.393021,13974.027873,0.064639,7.373969e+11,5.276927e+07,7.373657e+11,5.276687e+07,6.464235e-02
350,125832322,633657742,BMA,319,PA,780,True,1.000000,7400,6470,...,0.342977,0.999919,13965.637781,13966.771958,0.108243,1.234222e+12,8.836854e+07,1.234123e+12,8.836136e+07,1.082515e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13336,267153115,630982839,MDRN,395,NTS,651,True,0.126354,12840,5670,...,0.000103,0.015633,7.635580,488.411506,0.000002,1.392048e+07,8.106865e+04,6.190062e+05,1.267387e+03,9.930910e-05
13340,287950390,632186766,TRS,581,LSc,250,True,0.949175,5530,3300,...,0.119387,0.128298,60.788563,473.806196,0.005551,5.306314e+09,3.531805e+07,2.146933e+09,4.531248e+06,4.326461e-02
13556,286319739,631497971,NDB,596,AAA,23,True,0.000000,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,8.714411e+04,1.273160e+02,0.000000e+00,0.000000e+00,1.559621e-07
13570,581641279,635301707,PVH,38,LSr,258,True,0.000000,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.933751e+07,9.271531e+03,0.000000e+00,0.000000e+00,1.135763e-05


In [16]:
# For each node, count up # of escape edges it participates in
feeding_nodes['(outgoing edges)'] = feeding_nodes.apply(lambda row: outgoing_edges.loc[outgoing_edges['origin_id'] == row['structure-id']].shape[0], axis=1)
# For each node, count up # of nonescape edges it participates in
feeding_nodes['(incoming edges)'] = feeding_nodes.apply(lambda row: incoming_edges.loc[incoming_edges['target_id'] == row['structure-id']].shape[0], axis=1)
# For each node, count up # of internal edges it participates in
feeding_nodes['internal edges'] = feeding_nodes.apply(lambda row: internal_edges.loc[(internal_edges['origin_id'] == row['structure-id']) | (internal_edges['target_id'] == row['structure-id'])].shape[0], axis=1)

# Add up edges that communicate w/ outside feeding network
feeding_nodes['external edges'] = feeding_nodes['(outgoing edges)'] + feeding_nodes['(incoming edges)']
feeding_nodes.head()

,structure-id,structure-abbrev,structure-name,(outgoing edges),(incoming edges),internal edges,external edges
3,746,ORBvl,"Orbital area, ventrolateral part",59,3,13,62
4,596,NDB,Diagonal band nucleus,60,4,20,64
6,295,BLA,Basolateral amygdalar nucleus,76,7,11,83
7,872,DR,Dorsal nucleus raphe,44,7,5,51
9,693,VMH,Ventromedial hypothalamic nucleus,107,6,16,113


In [8]:
with pd.ExcelWriter('./feeding/feeding network 100 percent.xlsx') as writer:
    feeding_edges[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='Feeding Network', index=False)
    internal_edges[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='Internal Edges', index=False)
    external_edges[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='External Edges', index=False)
    feeding_nodes.to_excel(writer, sheet_name='Analysis', index=False)

In [18]:
# Get top 50% of feeding network
feeding_edges_50 = feeding_edges.iloc[:round(feeding_edges.shape[0] / 2)]
print('There are %d edges in the top 50%% volumes for feeding' % feeding_edges_50.shape[0])
feeding_edges_50.head()

There are 2386 edges in the top 50% intensities for feeding


,experiment_id,id,origin_abbrev,origin_id,target_abbrev,target_id,is_injection,max_voxel_density,max_voxel_x,max_voxel_y,...,normalized_projection_volume,projection_density,projection_energy,projection_intensity,projection_volume,sum_pixel_intensity,sum_pixels,sum_projection_pixel_intensity,sum_projection_pixels,volume
3,176886238,630101077,BST,351,PT,15,True,1.000000,5740,4620,...,0.117493,0.999045,18104.446380,18121.748533,0.065085,9.631659e+11,5.318124e+07,9.628168e+11,5.313046e+07,0.065147
6,127711803,633842902,CP,672,CEA,536,True,0.141788,6190,5410,...,0.000087,1.000000,17567.098039,17567.098039,0.000006,7.980640e+07,4.542947e+03,7.980640e+07,4.542947e+03,0.000006
10,603890605,641506553,TEa,541,TEa5,289,True,0.123172,8820,2340,...,0.000073,1.000000,17034.916362,17034.916362,0.000003,3.739768e+07,2.195354e+03,3.739768e+07,2.195354e+03,0.000003
13,609157409,641551186,PL,972,PL1,171,True,1.000000,3060,2930,...,0.007497,0.999882,16573.292891,16575.244074,0.001816,2.457081e+10,1.482373e+06,2.456780e+10,1.482198e+06,0.001816
14,176886238,630101459,BST,351,MTN,571,True,1.000000,5850,5070,...,0.159567,0.997939,16362.817350,16396.608046,0.088392,1.183618e+12,7.230566e+07,1.183124e+12,7.215665e+07,0.088574


In [17]:
# Find list of outgoing/incoming/internal edges
outgoing_edges_50 = feeding_edges_50.loc[~feeding_edges_50['target_id'].isin(feeding_nodes['structure-id'])]  # The target is not in the feeding network
incoming_edges_50 = feeding_edges_50.loc[(~feeding_edges_50['origin_id'].isin(feeding_nodes['structure-id'])) & (feeding_edges_50['target_id'].isin(feeding_nodes['structure-id']))]  # The origin isn't in the feeding network and the target is
internal_edges_50 = feeding_edges_50.loc[(feeding_edges_50['origin_id'].isin(feeding_nodes['structure-id'])) & (feeding_edges_50['target_id'].isin(feeding_nodes['structure-id']))]  # Both the origin and target are in the feeding network

external_edges_50 = pd.concat([outgoing_edges, incoming_edges])

In [20]:
feeding_nodes_50 = feeding_nodes
# For each node, count up # of escape edges it participates in
feeding_nodes_50['(outgoing edges)'] = feeding_nodes_50.apply(lambda row: outgoing_edges_50.loc[outgoing_edges_50['origin_id'] == row['structure-id']].shape[0], axis=1)
# For each node, count up # of nonescape edges it participates in
feeding_nodes_50['(incoming edges)'] = feeding_nodes_50.apply(lambda row: incoming_edges_50.loc[incoming_edges_50['target_id'] == row['structure-id']].shape[0], axis=1)
# For each node, count up # of internal edges it participates in
feeding_nodes_50['internal edges'] = feeding_nodes_50.apply(lambda row: internal_edges_50.loc[(internal_edges_50['origin_id'] == row['structure-id']) | (internal_edges_50['target_id'] == row['structure-id'])].shape[0], axis=1)

# Add up edges that communicate w/ outside feeding network
feeding_nodes_50['external edges'] = feeding_nodes_50['(outgoing edges)'] + feeding_nodes_50['(incoming edges)']
feeding_nodes_50.head()

,structure-id,structure-abbrev,structure-name,(outgoing edges),(incoming edges),internal edges,external edges
3,746,ORBvl,"Orbital area, ventrolateral part",52,1,10,53
4,596,NDB,Diagonal band nucleus,34,0,10,34
6,295,BLA,Basolateral amygdalar nucleus,38,3,8,41
7,872,DR,Dorsal nucleus raphe,25,2,3,27
9,693,VMH,Ventromedial hypothalamic nucleus,51,5,13,56


In [13]:
with pd.ExcelWriter('./feeding/feeding network 50 percent.xlsx') as writer:
    feeding_edges_50[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='Feeding Network', index=False)
    internal_edges_50[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='Internal Edges', index=False)
    external_edges_50[['origin_abbrev', 'origin_id', 'target_abbrev', 'target_id', 'projection_volume']].to_excel(writer, sheet_name='External Edges', index=False)
    feeding_nodes.to_excel(writer, sheet_name='Analysis', index=False)